In [212]:
import pandas as pd
import numpy as np
import sklearn
# import statsmodels.api as sm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [213]:
apple=pd.read_csv('apple.csv')
google=pd.read_csv('google.csv')

In [214]:
from sklearn import linear_model 
from sklearn.model_selection import train_test_split

In [215]:
apple['size']=apple['size'].apply(lambda x: x.replace(',','') if ',' in str(x) else x).apply(float)

In [216]:
##data transformation
apple['size']=np.log10(apple['size'])

In [217]:
apple['free']= np.where(apple['price']==0,1,0)#0
apple['less or equal $2']=np.where((apple['price']<=2) & (apple['price']>0),1,0)#1
apple['$2-$5']=np.where((apple['price']<=5) & (apple['price']>2),1,0)#3.5
apple['greater than $5']=np.where( (apple['price']>5),1,0)#12.22

In [218]:
# (n, bins, patches) = plt.hist(apple['size'], bins=20, label='hst')

# plt.show()
Nbins=5
hist, edges = np.histogram(apple['size'], bins=Nbins)

In [227]:
#train, test = train_test_split(apple, test_size = 0.5)
train=pd.read_csv('trainset.csv')
test=pd.read_csv("testset.csv")

In [228]:
x=np.zeros((4*Nbins,2))
count=0
price_list=[0,1,3.5,12.2228]
size_list=edges[0:-1]+(edges[1]-edges[0])/2
for p1 in range(4):
    for p2 in range(Nbins):
        x[count,0]=price_list[p1]
        x[count,1]=size_list[p2]
        count=count+1

In [229]:
count_list=np.zeros(4*Nbins)
rate_list=np.zeros(4*Nbins)

for p in range(len(train)):
    price=train.iloc[p]["price"]
    if price == 0:
        p1=0
    elif price<=2:
        p1=1
    elif price<=5:
        p1=2
    else:
        p1=3
    size=train.iloc[p]["size"]
    p2=int(np.floor((size-edges[0])/(edges[1]-edges[0])))
    if p2==Nbins:
        p2=Nbins-1
    count=Nbins*p1+p2
    count_list[count]= count_list[count]+1
    rate_list[count]=rate_list[count]+apple.iloc[p]["rating"]

nonzero=count_list!=0
x_train=x[nonzero]
y_train=rate_list[nonzero]/count_list[nonzero]

#### R^2 of train set

In [230]:
poly = sklearn.preprocessing.PolynomialFeatures(degree=2)
x_train2 = poly.fit_transform(x_train)

clf = linear_model.LinearRegression()
clf.fit(x_train2, y_train)
clf.score(x_train2, y_train)

0.5296412562122894

In [231]:
count_list=np.zeros(4*Nbins)
rate_list=np.zeros(4*Nbins)

for p in range(len(test)):
    price=test.iloc[p]["price"]
    if price == 0:
        p1=0
    elif price<=2:
        p1=1
    elif price<=5:
        p1=2
    else:
        p1=3
    size=test.iloc[p]["size"]
    p2=int(np.floor((size-edges[0])/(edges[1]-edges[0])))
    if p2==Nbins:
        p2=Nbins-1
    count=Nbins*p1+p2
    count_list[count]= count_list[count]+1
    rate_list[count]=rate_list[count]+apple.iloc[p]["rating"]

nonzero=count_list!=0
x_test=x[nonzero]
y_test=rate_list[nonzero]/count_list[nonzero]

#### R^2 of test set

In [232]:
x_test2 = poly.fit_transform(x_test)
clf.score(x_test2, y_test)

0.20138225831596468